Leaderboard
===========

Basic analysis and stats for private leaderboards.

Private leaderboards are available at the url in the form:  
https://adventofcode.com/2021/leaderboard/private/view/{leaderboard_id}.json

### Instructions  
1) Visit your private leaderboard at the json link  
2) Save the json to the json folder  
3) Update the path_leaderboard to point to the leaderboard json file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np

In [ ]:
path_leaderboard = 'json/1491207.json'
with open (path_leaderboard, 'r') as fh:
    data = json.load(fh)


In [ ]:
# Members
members = []
for member_id in data['members']:
    member_name = data['members'][member_id].get('name')
    member_local_score = data['members'][member_id].get('local_score')
    member = {
        "name": member_name,
        "id": member_id,
        "local_score": member_local_score
    }
    members.append(member)

df_members = pd.DataFrame.from_dict(members)
df_members.sort_values(['local_score'], ascending=False)

In [ ]:
member_count = df_members['id'].nunique()
member_count

In [ ]:
# Scores
scores = []

def seconds_since_release(day, dt):
    midnight = pd.Timestamp(year=2021, month=12, day=day, hour=5, minute=0, second=0, microsecond=0)
    seconds = (dt - midnight).seconds    
    return seconds

for member in data['members']:

    
    for day in data['members'][member].get('completion_day_level'):

        
        for star in data['members'][member].get('completion_day_level',{0: {}}).get(day, {}):
            score = ({
                'id': member, 
                'name': data['members'][member].get('name'),
                'day': int(day),
                'star': int(star),
                'ts': data['members'][member].get('completion_day_level').get(day).get(star)['get_star_ts'], 
                
            })
            score['dt'] = pd.to_datetime(score['ts'], unit='s')
            score['seconds_since_release'] = seconds_since_release(score['day'], score['dt'])
            score['elapsed_time'] = pd.Timedelta(seconds=score['seconds_since_release'])
            scores.append(score)

df_scores = pd.DataFrame.from_dict(scores)
df_scores

In [ ]:
day_max = df_scores['day'].max()
day_max

In [ ]:
# Calculate points
df_scores["part_rank"] = df_scores.groupby(['day', 'star'])["ts"].rank("dense", ascending=True).astype('int')
df_scores["points"] = df_scores['part_rank'].apply(lambda x: (member_count + 1) - x).astype('int')


In [ ]:
# Cumulative points over time
df_scores['c_points'] = df_scores.sort_values(['ts']).groupby(['id'])['points'].cumsum()

### Queries

In [ ]:
# Latest Scores
df_scores[df_scores['day'] == day_max].sort_values(['dt'])

In [ ]:
# Leaderboard
df_scores.groupby(df_scores['name'])['c_points'].max().sort_values(ascending=False)

In [ ]:
# What time does a member usually submit Part 2 by?
df_scores[df_scores['star'] == 2].groupby(['name'])['seconds_since_release'].mean().apply(lambda x: pd.Timedelta(seconds=int(x) + 5 * 60 * 60)).sort_values()

In [ ]:
# On average, how long does a member take between submiting P1 and P2 for a particular day?
df_pt_stars = pd.pivot_table(df_scores, 
                             values='seconds_since_release', 
                             index=['name', 'day'],
                             columns=['star'], 
                             aggfunc=np.max, fill_value=0).reset_index()

df_pt_stars['p1_to_p2_seconds'] = df_pt_stars[(2)] - df_pt_stars[(1)]

df_pt_stars['p1_to_p2_time'] = df_pt_stars['p1_to_p2_seconds'].apply(lambda x: pd.Timedelta(seconds=int(x)))
df_pt_stars

In [ ]:
df_p1_to_p2 = df_pt_stars[df_pt_stars[(2)]>0].groupby('name').agg({'p1_to_p2_seconds': ['count','mean', 'min', 'max'], 'day': ['max']}).reset_index()
df_p1_to_p2[('p1_to_p2_seconds', 'mean')] = df_p1_to_p2[('p1_to_p2_seconds', 'mean')].astype('int')
df_p1_to_p2['time_mean'] = df_p1_to_p2[('p1_to_p2_seconds', 'mean')].apply(lambda x:  pd.Timedelta(seconds=int(x)))
df_p1_to_p2.sort_values(('p1_to_p2_seconds','mean'))

In [ ]:
# What did do today's split timings look like?
df_pt_stars[df_pt_stars['day'] == 8].sort_values('p1_to_p2_time', ascending=True)

In [ ]:
# Which was a member's best P1 to P2 submission?
df_pt_stars[df_pt_stars[(2)]>0].sort_values('p1_to_p2_seconds',ascending = True).groupby(['name']).head(1)
